# Analyse des données de transport

## Objectifs

Ce notebool a pour objectif d'explorer et d'analyser un historique de tickets de transports:
* Analyser les prix et durée des trajets
* comparer les modes de transport selon la distance
* identifier des tendances et incohérences dans les données
* proposer des analyses et visualisations pertinentes

## Outils utilisés
* Python
* Pandas
* NumPy
* Matplotlib / Seaborn
* Plotly, Scikit-learn
